# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [83]:
import hvplot.pandas
import pandas as pd
import requests

from api_keys import geoapify_key

In [84]:
city_data_df = pd.read_csv("output_data/cities.csv")

city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,0,jinchang,38.4953,102.1739,267.99,36,100,1.23,CN
1,1,busselton,-33.6500,115.3333,285.93,79,6,6.58,AU
2,2,qaanaaq,77.4840,-69.3632,249.49,66,53,3.95,GL
3,3,rikitea,-23.1203,-134.9692,299.43,76,11,6.64,PF
4,4,abalak,58.1286,68.5944,262.73,98,96,1.86,RU


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [85]:
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    scale=1,
    alpha=0.5,
    color="City",
    tiles="OSM",
    frame_width=700,
    frame_height=500
)
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [88]:
city_temp_df = city_data_df.loc[city_data_df["Max Temp"] > 300 ]
city_temp_df.dropna()
city_temp_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
6,6,hithadhoo,-0.6000,73.0833,300.22,77,31,4.71,MV
13,13,lazaro cardenas,17.9583,-102.2000,301.29,52,3,4.37,MX
19,19,puerto ayora,-0.7393,-90.3518,303.13,92,85,2.24,EC
22,22,kapaa,22.0752,-159.3190,300.64,77,0,2.57,US
32,32,limon,10.0000,-83.0333,301.09,69,0,3.09,CR


### Step 3: Create a new DataFrame called `hotel_df`.

In [89]:
hotel_df = city_temp_df[["City","Country","Lat","Lng","Humidity"]].copy()
hotel_df["Hotel Name"] = ''
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
6,hithadhoo,MV,-0.6000,73.0833,77,
13,lazaro cardenas,MX,17.9583,-102.2000,52,
19,puerto ayora,EC,-0.7393,-90.3518,92,
22,kapaa,US,22.0752,-159.3190,77,
32,limon,CR,10.0000,-83.0333,69,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [92]:
radius = 10000
params = {
    "filter": "",
    "bias": "",
    "apiKey": geoapify_key,
    "type": "accommodation.hotel"
}

print("Starting hotel search")

for index, row in hotel_df.iterrows():
    longitude = row["Lng"]
    latitude = row["Lat"]

    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
   
    base_url = "https://api.geoapify.com/v2/places"

    name_address = requests.get(base_url, params=params)
    
    name_address = name_address.json()

    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

hotel_df.head()

Starting hotel search
hithadhoo - nearest hotel: Pebbles Inn
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
kapaa - nearest hotel: Pono Kai Resort
limon - nearest hotel: Hotel NG
butaritari - nearest hotel: No hotel found
ewa beach - nearest hotel: Hampton Inn & Suites Oahu/Kapolei
tautira - nearest hotel: No hotel found
arraial do cabo - nearest hotel: No hotel found
hilo - nearest hotel: Dolphin Bay Hotel
kavieng - nearest hotel: Kavieng Hotel
kirakira - nearest hotel: No hotel found
nhulunbuy - nearest hotel: No hotel found
bafoulabe - nearest hotel: No hotel found
arica - nearest hotel: Hotel Andalucía 
rolante - nearest hotel: Pousada Rolantchê
varzea grande - nearest hotel: Slaviero Slim Aeroporto
thinadhoo - nearest hotel: No hotel found
charters towers - nearest hotel: No hotel found
vila velha - nearest hotel: Hotel Prainha
vaini - nearest hotel: Keleti International Resort
avarua - nearest hotel: Para

,City,Country,Lat,Lng,Humidity,Hotel Name
6,hithadhoo,MV,-0.6000,73.0833,77,Pebbles Inn
13,lazaro cardenas,MX,17.9583,-102.2000,52,Hotel Sol del Pacífico
19,puerto ayora,EC,-0.7393,-90.3518,92,Hostal La Mirada De Solitario George
22,kapaa,US,22.0752,-159.3190,77,Pono Kai Resort
32,limon,CR,10.0000,-83.0333,69,Hotel NG


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [100]:
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    scale=1,
    color="City",
    tiles="OSM",
    hover_cols = ["Hotel Name"],
    frame_width=700,
    frame_height=500
)
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)